Заглушка к ДЗ 3 (сделаю на выходных)

1. Прочитайте базу my_PJME_MW.csv и сделайте ее перрасчет (ресемплинг) в представление по 1 месяцу.


2.Подберите параметры модели SARIMAX для этого ряда.


3.Сделайте вывод о том, как изменилась модель по сравнению с недельной.


4. Возмите вариант 1 или 2


- Вариант 1) - Постройте ACF, PACF и модель ARIMA для BRENT (недельный ресемплинг)

- Вариант 2) - Постройте ACF, PACF и модель ARIMA для СOVID_19 (по любой стране кроме Китая на дневных графиках )


5.Оцените результат моделирования


6.Сделайте предположение о моделях, которые лучше работают с техниками регрессии на основе AR I MA.